In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 

Preparing a dataframe

In [10]:
path="data/"
news_path=path+"nytimes_news_articles.txt"

In [11]:
def parse_url(url):
    date=url[28:38]
    rest_of_url=url[39:]
    category_slash=rest_of_url.find('/')
    category=rest_of_url[:category_slash]
    rest_of_url=rest_of_url[category_slash+1:]
    subcategory="NIL"
    if(rest_of_url.find('/')!=-1):
        subcategory_slash=rest_of_url.find('/')
        subcategory=rest_of_url[:subcategory_slash]    
        rest_of_url=rest_of_url[subcategory_slash+1:]    
    title=rest_of_url[:-5]
    title=title.replace('-',' ')
    return date,category,subcategory,title
    
    
    
    

In [12]:
URL=[]
Date=[]
Article=[]
Category=[]
Subcategory=[]
Title=[]
with open(news_path) as news_file:
    line=news_file.readline()
    while line:
        URL.append(line[5:])
        date_,category_,subcategory_,title_=parse_url(line)
        Date.append(date_)
        Category.append(category_)
        Subcategory.append(subcategory_)
        Title.append(title_)
        #print(title_)
        article_=""
        line=news_file.readline()
        while line and line[:4]!="URL:":
            article_+=line+" "
            line=news_file.readline()
        Article.append(article_[2:].replace('\n',''))    
            

In [13]:
news_df=pd.DataFrame()
news_df['URL']=URL
news_df['Date']=Date
news_df['Category']=Category
news_df['Subcategory']=Subcategory
news_df['Article']=Article
news_df['Title']=Title


In [14]:
news_df.head()

,URL,Date,Category,Subcategory,Article,Title
0,http://www.nytimes.com/2016/06/30/sports/baseb...,2016/06/30,sports,baseball,WASHINGTON — Stellar pitching kept the Mets af...,washington nationals max scherzer baffles mets...
1,http://www.nytimes.com/2016/06/30/nyregion/may...,2016/06/30,nyregion,NIL,Mayor Bill de Blasio’s counsel and chief legal...,mayor de blasios counsel to leave next month t...
2,http://www.nytimes.com/2016/06/30/nyregion/thr...,2016/06/30,nyregion,NIL,In the early morning hours of Labor Day last y...,three men charged in killing of cuomo administ...
3,http://www.nytimes.com/2016/06/30/nyregion/tek...,2016/06/30,nyregion,NIL,It was the Apple Store in New York City before...,tekserve precursor to the apple store to close...
4,http://www.nytimes.com/2016/06/30/sports/olymp...,2016/06/30,sports,olympics,OMAHA — The United States Olympic swimming tri...,once at michael phelpss feet and still chasing...


In [15]:
category_set=set(Category)
print(len(category_set))
for category in category_set:
    print(category)

28
arts
science
travel
magazine
world
technology
t-magazine
fashion
nyregion
jobs
upshot
education
universal
theater
nytnow
sports
us
books
style
dining
automobiles
insider
pageoneplus
health
realestate
movies
business
your-money


In [16]:
subcategory_set=set(Subcategory)
print(len(subcategory_set))
for category in subcategory_set:
    print(category)

53
weddings
food
tennis
smallbusiness
travel
americas
europe
what-in-the-world
soccer
design
cycling
energy-environment
australia
ncaafootball
events
elections
mens-style
commercial
fashion
review
middleeast
international
africa
football
asia
baseball
art
student-loans
auto-insurance
golf
sailing
artsspecial
NIL
music
television
horse-racing
economy
ncaabasketball
politics
personaltech
wheels
autoracing
cricket
dealbook
olympics
autoreviews
media
rugby
basketball
entertainment
ko
hockey
dance


Stopwords

In [21]:
stopwords_path=path+"stopwords.txt"

In [22]:
stopwords=[]
with open(stopwords_path) as stopwords_file:
    line=stopwords_file.readline().replace('\n','').replace('-',' ')
    while line:
        stopwords.append(line)
        line=stopwords_file.readline()[:-1].replace('\n','').replace('-',' ')

In [4]:
np.save(path+'stopwords',stopwords)


NameError: name 'stopwords' is not defined

In [6]:
stopwords=np.load(path+'stopwords.npy')

In [7]:
freq_map={}

In [14]:
ps=PorterStemmer()
for article in news_df['Article']:
    word_list=word_tokenize(article)

    for word in word_list:
        if word not in stopwords:
            word_s=ps.stem(word)
            if word_s in freq_map:
                freq_map[word_s]+=1
            else:
                freq_map[word_s]=1
    

In [15]:
freq_map=np.load(path+'freq_map.npy')

In [18]:
len(freq_map)

111576

In [19]:
number_of_articles=len(news_df)

In [ ]:
vector_space_table=np.zeros(shape=(number_of_articles,len(freq_map)))

In [ ]:
word_to_index={}
index_to_word={}
index=0
for key in freq_map:
    word_to_index[key]=index
    index_to_word[index]=key
    index+=1  

In [ ]:
for i in range(number_of_articles):
  article=news_df['Article'][i]
  word_list=word_tokenize(article)
  article_map={}
  for word in word_list:
    word=word.replace('\n','').replace('-',' ')
    if len(word)>2:
      if word.isalpha():
        if word not in stopwords:
          word_s=ps.stem(word)
          if word_s not in stopwords:
            try :
              float(word_s.replace(',',''))
              continue
            except :  
              if word_s in article_map:
                article_map[word_s]+=1
              else:
                article_map[word_s]=1
  for word in article_map:
    map_index=word_to_index[word]
    freq_art=article_map[word]
    freq_total=freq_map[word]
    vector_space_table[i][map_index]=freq_art*np.log(number_of_articles/number_of_occ[map_index])

In [ ]:
def low_rank_k(mat,num):
    u,s,vh=np.linalg.svd(mat, full_matrices= False)
    u = u[:,:num]
    vh = vh[:num,:]
    s = s[:num]
    s = np.diag(s)
    #my_low_rank = np.dot(np.dot(u,s),vh)
    return u,s,vh

In [ ]:
np.save(path+'u',u)
np.save(path+'s',s)
np.save(path+'vh',vh)

In [ ]:
for i in range(70):
  topic_vector=u[:,i]
  word_list=[]
  for j in range(len(freq_map)):
    current_vector=vector_space_table[:,j]
    val=np.dot(vector_space_table[:,j],u[:,i])
    word_list.append((val,j))
  print("Topic "+str(i+1)+" : ")
  word_list.sort(key=lambda x: x[0],reverse=True)
  for i in range(10):
    print(index_to_word[word_list[i][1]]+",",end='')
  print('\n')  